In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from sklearn.cross_validation import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras import layers
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Activation,Dropout
from sklearn.preprocessing import StandardScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train=pd.read_csv('/kaggle/input/motorbike-cost/train.csv')
test=pd.read_csv('/kaggle/input/motorbike-cost/test.csv')
train_y=train['price']
train.pop('price')
train.pop('model_name')
train.pop('bike_id')
test.pop('model_name')
test.pop('bike_id')
# train.pop('model_year')
# train.pop('owner')
j=0
for i in train['kms_driven']:
    if(type(i)==str):
        arr=i.split(' ')
#         print(arr)
        if (arr[-1]=='Km' and arr[0]!='Yes'):
            train['kms_driven'] = train['kms_driven'].replace(i,float(arr[0]))
            continue
        if (len(arr)==1 and arr[0].lower()!='yes'):
            train['kms_driven'] = train['kms_driven'].replace(i,float(arr[0]))
        if(arr[0]=='Mileage'):
            train['kms_driven']=train['kms_driven'].replace(i,0)
        else:
            train['kms_driven']=train['kms_driven'].replace(i,0)
# train['kms_driven'].reset_index(drop=True)
for i in train['mileage']:
    if(type(i)==str):
        arr=i.split(' ')
        #print(arr)
        if (len(arr)==2):
            train['mileage'] = train['mileage'].replace(i,0)
            continue
        if (arr[2].lower()=='kmpl'):
            arr1=arr[1].split('-')
            if len(arr1)==2:
                train['mileage'] = train['mileage'].replace(i,(float(arr1[0])+float(arr1[1]))/2)
            else:
                train['mileage'] = train['mileage'].replace(i,float(arr[1]))
        if(arr[2].lower()=='kms'):
            train['mileage'] = train['mileage'].replace(i,float(arr[1]))
        else:
             train['mileage'] = train['mileage'].replace(i,0)
                             
for i in train['power']:
    if(type(i)==str):
        arr=i.split(' ')
        if (arr[-1].lower()=='bhp'):
            train['power'] = train['power'].replace(i,float(arr[0]))
        if (arr[-1].lower()=='rpm' or arr[-1].lower()=='kw'):
            train['power'] = train['power'].replace(i,float(arr[0]))
        if(arr[-1].lower()=='ps'):
            train['power'] = train['power'].replace(i,float(arr[0])*0.986)
        if(len(arr)==4 ):
            train['power'] = train['power'].replace(i,float(arr[0])*1.34)
        if(len(arr)==3):
            train['power'] = train['power'].replace(i,6.15*1.34)
        else:
            train['power'] = train['power'].replace(i,10)
# indexAge=train[train['kms_driven'].eq(0) | train['mileage'].eq(0) | train['power'].eq(0.0)].index
# train.drop(indexAge, inplace=True)
# train_y.drop(indexAge, inplace=True)
for i in test['kms_driven']:
    if(type(i)==str):
        arr=i.split(' ')
#         print(arr)
        if (arr[-1]=='Km'):
            test['kms_driven'] = test['kms_driven'].replace(i,float(arr[0]))
            continue
        if (len(arr)==1 and arr[0].lower()!='yes'):
            test['kms_driven'] = test['kms_driven'].replace(i,float(arr[0]))
        if(arr[0]=='Mileage'):
            test['kms_driven']=test['kms_driven'].replace(i,0)
        else:
            test['kms_driven']=test['kms_driven'].replace(i,0)
# train['kms_driven'].reset_index(drop=True)
for i in test['mileage']:
    if(type(i)==str):
        arr=i.split(' ')
        #print(arr)
        if (len(arr)==2):
            test['mileage'] = test['mileage'].replace(i,0)
            continue
        if (arr[2].lower()=='kmpl'):
            arr1=arr[1].split('-')
            if len(arr1)==2:
                test['mileage'] = test['mileage'].replace(i,(float(arr1[0])+float(arr1[1]))/2)
            else:
                test['mileage'] = test['mileage'].replace(i,float(arr[1]))
        if(arr[2].lower()=='kms'):
            test['mileage'] = test['mileage'].replace(i,float(arr[1]))
        else:
             test['mileage'] = test['mileage'].replace(i,0)
                             
for i in test['power']:
    if(type(i)==str):
        arr=i.split(' ')
        if (arr[-1].lower()=='bhp'):
            test['power'] = test['power'].replace(i,float(arr[0]))
        if (arr[-1].lower()=='rpm' or arr[-1].lower()=='kw'):
            test['power'] = test['power'].replace(i,float(arr[0]))
        if(arr[-1].lower()=='ps'):
            test['power'] = test['power'].replace(i,float(arr[0])*0.986)
        if(len(arr)==4 ):
            test['power'] = test['power'].replace(i,float(arr[0])*1.34)
        if(len(arr)==3):
            test['power'] = test['power'].replace(i,6.15*1.34)
        else:
            test['power'] = test['power'].replace(i,10)
# indexAge=test[test['kms_driven'].eq(0) | test['mileage'].eq(0) | test['power'].eq(0.0)].index
# test.drop(indexAge, inplace=True)
train=train.replace(0,np.nan)
test=test.replace(0.0,np.nan)
train['location'].fillna('unknown',inplace=True)
train['owner'].fillna('unknown',inplace=True)
le=LabelEncoder()
label = le.fit_transform(train['owner'])
train.drop("owner", axis=1, inplace=True)
train["owner"] = label
label = le.fit_transform(test['owner'])
test.drop("owner", axis=1, inplace=True)
test["owner"] = label
label = le.fit_transform(train['location'])
train.drop("location", axis=1, inplace=True)
train["location"] = label
label = le.fit_transform(train['model_year'])
train.drop("model_year", axis=1, inplace=True)
train["model_year"] = label
label = le.fit_transform(test['model_year'])
test.drop("model_year", axis=1, inplace=True)
test["model_year"] = label
label = le.fit_transform(test['location'])
test.drop("location", axis=1, inplace=True)
test["location"] = label
train=train.interpolate(method='linear',limit_direction='forward')
test=test.interpolate(method='linear',limit_direction='backward')
train['kms_driven']=tf.convert_to_tensor(train['kms_driven'].astype('float32'))
train['owner']=tf.convert_to_tensor(train['owner'].astype('float32'))
train['power']=tf.convert_to_tensor(train['power'].astype('float32'))
train['model_year']=tf.convert_to_tensor(train['model_year'].astype('float32'))
train['location']=tf.convert_to_tensor(train['location'].astype('float32'))
train=np.asarray(train).astype(np.float32)
train_y=np.asarray(train_y).astype(np.float32)
test=np.asarray(test).astype(np.float32)
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.fit_transform(test)
print(test)
model = LogisticRegression(random_state=2,solver='lbfgs', max_iter=1000).fit(train,train_y)
arr=model.predict(test)
print(arr)

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/motorbike-cost/sample_submission.csv
/kaggle/input/motorbike-cost/train.csv
/kaggle/input/motorbike-cost/test.csv


2022-11-02 15:25:24.725450: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


[[-0.3483269   1.2104722  -0.05589373 -0.38088477  0.9609629  -0.56983787]
 [ 0.02398302  1.2104722  -0.3817713  -0.38088477 -1.4623407  -1.2585173 ]
 [ 0.85847074 -0.31360614 -0.11020661 -0.38088477 -2.5393648  -1.2585173 ]
 ...
 [-0.540901   -0.2831246   0.63659626 -0.38088477  1.2302189  -0.30967012]
 [ 0.71724975  1.2104722  -0.74566793 -0.38088477 -1.4623407  -1.641117  ]
 [-0.3765711  -0.1916799  -0.26635626 -0.38088477 -0.92382884 -0.56983787]]
[ 45000.  25000.  70000. ... 125000.  25000.  80000.]


In [2]:
import csv
filename = "motorbike.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(['bike_id','price'])
    for i in range(len(arr)):
        csvwriter.writerow([i,int(arr[i])])
print(pd.read_csv(filename))
pd.read_csv(filename).to_csv('/kaggle/working/motorbike.csv')

      bike_id    price
0           0    45000
1           1    25000
2           2    70000
3           3   165000
4           4    75000
...       ...      ...
1960     1960    90000
1961     1961  1500000
1962     1962   125000
1963     1963    25000
1964     1964    80000

[1965 rows x 2 columns]
